In [1]:
%matplotlib
import matplotlib as plt
import torch
import numpy as np
import sys
import ipdb
sys.path.append('..')
import d2lzh_pytorch as d2l

Using matplotlib backend: agg


In [5]:
n_train, n_test, true_w, true_b = 100, 100, [1.2, -3.4, 5.6], 5
features = torch.randn((n_train + n_test, 1))
poly_features = torch.cat((features, torch.pow(features, 2), torch.pow(features, 3)), 1)
labels = (true_w[0] * poly_features[:, 0] + true_w[1] * poly_features[:, 1] + true_w[2] * poly_features[:, 2] + true_b)
labels += torch.tensor(np.random.normal(0, 0.01, labels.size()), dtype=torch.float)

In [8]:
def semilogy(x_vals, y_vals, x_label, y_label, x2_vals=None, y2_vals=None, legend=None, figsize=(3.5, 2.5)):
    d2l.set_figsize(figsize)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.semilogy(x_vals, y_vals)
    if x2_vals and y2_vals:
        plt.semilogy(x2_vals, y2_vals， linestyle='')
        plt.legend(legend)


tensor([[-1.2208],
        [ 1.2324]]) tensor([[-1.2208,  1.4904, -1.8196],
        [ 1.2324,  1.5188,  1.8718]]) tensor([-11.7236,  11.7864])


In [ ]:
num_epochs, loss = 100, torch.nn.MSEL